In [1]:
# Dependencies
import requests
import json
#import wbdata
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# import csv data files
education_file = "Resources/education.csv"
childmarriage_file = "Resources/childmarriage.csv"

# create dataframes for each
education_file_df = pd.read_csv(education_file, encoding="ISO-8859-1")
childmarriage_file_df = pd.read_csv(childmarriage_file, encoding="ISO-8859-1")
countries_df = pd.read_html('https://developers.google.com/public-data/docs/canonical/countries_csv')

#education_file_df.head()
#childmarriage_file_df.head()
#countries_df[0]

FileNotFoundError: [Errno 2] File b'Resources/education.csv' does not exist: b'Resources/education.csv'

In [ ]:
filtered_education = education_file_df.filter(["Indicator"== "Youth literacy rate, population 15-24 years, both sexes (%)",
                                              "LOCATION", "Country","TIME", "Value"])
filtered_education.head()                       

In [ ]:
childmarriage_file_df.head()
clean_childmarriage=childmarriage_file_df[['Geographic Area','Indicator','Sex','Time Period','Current Age',
                           'Observation Value','Unit of Measure','The period of time for which data are provided']]
clean_childmarriage = clean_childmarriage.rename(columns={"Geographic Area":"Country"})
clean_childmarriage.head()

In [ ]:
countries_df[0]
clean_countries = countries_df[0].rename(columns={"name":"Country","country":"Abbr"})
clean_countries.head()

In [ ]:
# make country lowercase
clean_childmarriage["Country"] = clean_childmarriage["Country"].map(lambda x: x.lower())
clean_countries["Country"] = clean_countries["Country"].map(lambda x: x.lower())
filtered_education["Country"] = filtered_education["Country"].map(lambda z: z.lower())

In [ ]:
childmarriage_final = pd.merge(clean_childmarriage, clean_countries, how="left", on="Country")
childmarriage_final = childmarriage_final.drop(columns=["The period of time for which data are provided","Abbr"])
childmarriage_final
childmarriage_final.head()

In [ ]:
education_final = pd.merge(filtered_education, clean_countries, how="left", on="Country")
education_final = education_final.drop(columns=["LOCATION","Abbr"])
education_final.head()

In [ ]:
# from education_final dataset, group each country and get the mean of values
new_education_final=education_final.groupby(["Country"])
education_grouped_country=new_education_final["Value"].mean()
education_grouped_country_df=pd.DataFrame(education_grouped_country)
education_grouped_country_df

In [ ]:
# childmarriage_final dataset, group each country and get the mean of values
new_childmarriage_final=childmarriage_final.groupby(["Country"])
childmarriage_grouped_country=new_childmarriage_final["Observation Value"].mean()
childmarriage_grouped_country_df=pd.DataFrame(childmarriage_grouped_country)
childmarriage_grouped_country_df

In [ ]:
# Merge data from childmarriage_grouped_country_df and education_grouped_country_df to compare
childmarriage_and_education = pd.merge(childmarriage_grouped_country_df, education_grouped_country_df, how="inner", on="Country")
childmarriage_and_education
childmarriage_and_education = childmarriage_and_education.rename(columns={"Observation Value":"Overall Averaged Child Marriage Value",
                                                                         "Value":"Averaged Education Value"})
childmarriage_and_education

In [ ]:
print (childmarriage_and_education["Overall Averaged Child Marriage Value"].median())


In [ ]:
# Correlation of Education with child marriage
x_values = childmarriage_and_education['Averaged Education Value']
y_values = childmarriage_and_education['Overall Averaged Child Marriage Value']
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)
regress_values = x_values * slope + intercept
#print(regress_values)
line_eq = "y = " + str(round(slope,5)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Averaged Education Value")
plt.ylabel("Overall Averaged Child Marriage Value")
print(line_eq)
print(f"The r-squared is: {rvalue}")
plt.show()

In [ ]:
# Boxplot for child marriage 
fig1, ax1 = plt.subplots()
ax1.set_title('Child Marriage')
ax1.set_ylabel('Averaged Child Marriage Value')
value=childmarriage_and_education["Overall Averaged Child Marriage Value"]
ax1.boxplot(value)
plt.show()
# Give quartile calculations
quartiles = value.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
# Print result for child marriage
print(f"The lower quartile of averaged child marriage value is: {lowerq}")
print(f"The upper quartile of averaged child marriage value is: {upperq}")
print(f"The interquartile range of averaged child marriage value is: {iqr}")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")
print(f"There are outliers for child marriage:")
outliers=value.loc[value > 33.92916666666667]
print(outliers)

In [ ]:
# Boxplot for child education 
fig1, ax1 = plt.subplots()
ax1.set_title('Child Education')
ax1.set_ylabel('Averaged Child Education')
value=childmarriage_and_education["Averaged Education Value"]
ax1.boxplot(value)
plt.show()
# Give quartile calculations
quartiles = value.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
# Print result for Campomulin
print(f"The lower quartile of averaged child education is: {lowerq}")
print(f"The upper quartile of averaged child education is: {upperq}")
print(f"The interquartile range of averaged child education is: {iqr}")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")
outliers=value.loc[value < 29.800143026315816]
print(f"There are outliers for child education:")
print(outliers)

In [ ]:
# Question 3: What is the geographical impact on child marriage and education? (heat map one for education and one for child marriage)

In [ ]:
# Question 4: Is one gender more susceptible to child marriage than the other? (pie graph)

In [ ]:
# Conclusion: our null hypothesis is that education has no impact on child marriage rates.

In [ ]:
list1 = list(clean_childmarriage["Country"].unique())
list2 = list(clean_countries["Country"].unique())

print(len(list1), len(list2))

In [ ]:
clean_childmarriage["Country"] = clean_childmarriage["Country"].map(lambda x: x.lower())
clean_childmarriage.head()

In [ ]:
clean_countries["Country"] = clean_countries["Country"].map(lambda x: x.lower())
clean_countries.head()

In [ ]:
df_merge = pd.merge(clean_childmarriage, clean_countries, how="left", on="Country")
df_merge.head()

In [ ]:
df_merge["Country"].unique()

In [ ]:
df_merge.loc[df_merge.isnull().any(axis=1)]

In [ ]:
df_merge.dtypes

In [ ]:
df_merge = df_merge.rename(columns={
    "Time Period":"TIME"
})
df_merge = df_merge.drop(columns=['The period of time for which data are provided'])
df_merge.head(1)


In [ ]:
filtered_education.head(1)

In [ ]:
filtered_education = filtered_education.drop(columns="LOCATION")
filtered_education.head(1)

In [ ]:
filtered_education["Country"] = filtered_education["Country"].map(lambda z: z.lower())
filtered_education.head(1)

In [ ]:
df_merge.shape

In [ ]:
filtered_education.shape

In [ ]:
df_merge[["Country", "TIME"]].nunique()

In [ ]:
filtered_education[["Country", "TIME"]].nunique()

In [ ]:
quick_merge_df = pd.merge(df_merge, filtered_education, on=["Country", "TIME"])
quick_merge_df

In [ ]:
quick_merge_df

In [ ]:
quick_merge_df = pd.merge(df_merge, filtered_education, on=["Country", "TIME"], how="outer")
quick_merge_df.shape

In [ ]:
quick_merge_df.loc[quick_merge_df.isnull().any(axis=1)]

In [ ]:
filtered_education.dtypes

In [ ]:
df_merge.shape

In [ ]:
df_merge = df_merge.dropna(axis=0)
df_merge.shape

In [ ]:
df_merge.shape

In [ ]:
df_merge = df_merge.dropna(axis=0)
df_merge.shape

In [ ]:
clean_countries["Country"].unique()

In [ ]:
# Merge the child marriage dataframe with the education dataframe
new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])


merge_df=pd.merge(pd.merge(clean_childmarriage,clean_countries,on='Country',),filtered_education,on='Country')
merge_df.dropna()
merge_df = merge_df.drop(columns=['The period of time for which data are provided'])
merge_df
#clean_df=merge_df.drop_duplicates('SN')

In [ ]:
print(merge_df["views"].max())
print(merge_df["views"].min())

In [ ]:
# Create bins in which to place values based upon TED Talk views
bins = [0, 199999, 399999, 599999, 799999, 999999,
        1999999, 2999999, 3999999, 4999999, 50000000]

# Create labels for these bins
group_labels = ["0 to 199k", "200k to 399k", "400k to 599k", "600k to 799k", "800k to 999k", "1mil to 2mil",
                "2mil to 3mil", "3mil to 4mil", "4mil to 5mil", "5mil to 50mil"]

pd.cut(ted_df["views"], bins, labels=group_labels).head()

# Place the data series into a new column inside of the DataFrame
ted_df["View Group"] = pd.cut(ted_df["views"], bins, labels=group_labels)
ted_df.head()